In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.3 MB/s eta 0:00:00


In [2]:
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable, get_cmap
import random

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [45]:
!gdown 1sNUKLwdllYqZIGna4IiiNq-pw4hW3q6y

Downloading...
From (original): https://drive.google.com/uc?id=1sNUKLwdllYqZIGna4IiiNq-pw4hW3q6y
From (redirected): https://drive.google.com/uc?id=1sNUKLwdllYqZIGna4IiiNq-pw4hW3q6y&confirm=t&uuid=66a9df2d-daa8-4d1c-886d-b590b7869067
To: /content/endfinal_individualfin.csv
100% 138M/138M [00:02<00:00, 48.2MB/s]


In [46]:
import pandas as pd

# Load the data
df = pd.read_csv("/content/endfinal_individualfin1.csv")

# Step 1: Add a column for total area of all elements
area_columns = [col for col in df.columns if "area" in col and col != "O_area"]
df["Total_area"] = df[area_columns].sum(axis=1)

# Step 2: Drop all uncertainty columns
uncertainty_columns = [col for col in df.columns if "uncertainity" in col]
df.drop(columns=uncertainty_columns, inplace=True)

# Step 3: Drop the oxygen-related column
df.drop(columns=["O_area"], inplace=True, errors="ignore")

# Step 4: Fill NaN values with 0
df.fillna(0, inplace=True)

# Step 5: Add columns for ratio of each element's area to total area
for col in area_columns:
    if col != "O_area":  # Ensure "O_area" is excluded
        df[f"{col}_ratio"] = df[col] / df["Total_area"]

# Step 6: Create a new DataFrame with only total area and element-area-ratio columns along with latitudes and longitudes
lat_lon_columns = [col for col in df.columns if "lat" in col or "lon" in col]
ratio_columns = [col for col in df.columns if "_ratio" in col]
new_df = df[lat_lon_columns + ["Total_area"] + ratio_columns]

# Save or display the resulting DataFrame
#new_df.to_csv("processed_data.csv", index=False)
new_df.head()


,V0_lat,V0_lon,V1_lat,V1_lon,V2_lat,V2_lon,V3_lat,V3_lon,Total_area,Fe_area_ratio,Na_area_ratio,Mg_area_ratio,Al_area_ratio,Si_area_ratio,Ca_area_ratio,Ti_area_ratio,Mn_area_ratio
0,74.8951,-141.7370,73.8060,-141.5740,73.8182,-139.1530,74.9082,-139.1430,462.915804,0.103721,0.107867,0.026593,0.686842,0.006900,0.005601,0.060511,0.001966
1,67.4519,40.2033,66.2786,40.2092,66.2690,42.1074,67.4418,42.1921,205.613010,0.084217,0.042430,0.396487,0.196951,0.242146,0.001060,0.004524,0.032185
2,65.8145,40.2049,64.6371,40.2094,64.6281,42.0018,65.8051,42.0765,215.594182,0.215140,0.074796,0.251008,0.216873,0.134231,0.019969,0.014770,0.073214
3,64.5870,40.2050,63.4069,40.2089,63.3982,41.9309,64.5779,41.9990,264.081858,0.052671,0.131113,0.242000,0.261851,0.198091,0.109614,0.001648,0.003012
4,63.3600,40.2043,62.1770,40.2075,62.1686,41.8657,63.3513,41.9283,226.717344,0.084094,0.001960,0.387413,0.166336,0.327291,0.027735,0.002913,0.002260


In [47]:
# Normalizing the ratio columns
ratio_columns = ['Na_area_ratio', 'Mg_area_ratio', 'Al_area_ratio', 'Ca_area_ratio', 'Ti_area_ratio', 'Mn_area_ratio', 'Fe_area_ratio']

# Calculate the sum of all (Element/Si Ratios) for each row
row_sums = new_df[ratio_columns].sum(axis=1)

# Normalize each element using the specified formula
new_df[ratio_columns] = new_df[ratio_columns].div(row_sums, axis=0)
new_df

<ipython-input-47-4eafdbde9094>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[ratio_columns] = new_df[ratio_columns].div(row_sums, axis=0)


,V0_lat,V0_lon,V1_lat,V1_lon,V2_lat,V2_lon,V3_lat,V3_lon,Total_area,Fe_area_ratio,Na_area_ratio,Mg_area_ratio,Al_area_ratio,Si_area_ratio,Ca_area_ratio,Ti_area_ratio,Mn_area_ratio
0,74.8951,-141.7370,73.8060,-141.5740,73.8182,-139.1530,74.9082,-139.1430,462.915804,0.104442,0.108616,0.026777,0.691614,6.899995e-03,5.639695e-03,6.093111e-02,1.979451e-03
1,67.4519,40.2033,66.2786,40.2092,66.2690,42.1074,67.4418,42.1921,205.613010,0.111126,0.055987,0.523171,0.259879,2.421458e-01,1.398886e-03,5.969466e-03,4.246917e-02
2,65.8145,40.2049,64.6371,40.2094,64.6281,42.0018,65.8051,42.0765,215.594182,0.248495,0.086392,0.289925,0.250497,1.342312e-01,2.306543e-02,1.705956e-02,8.456512e-02
3,64.5870,40.2050,63.4069,40.2089,63.3982,41.9309,64.5779,41.9990,264.081858,0.065682,0.163501,0.301780,0.326535,1.980912e-01,1.366909e-01,2.055383e-03,3.756327e-03
4,63.3600,40.2043,62.1770,40.2075,62.1686,41.8657,63.3513,41.9283,226.717344,0.125008,0.002913,0.575899,0.247262,3.272906e-01,4.122873e-02,4.330467e-03,3.358896e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315686,66.0878,154.2270,64.8541,154.3890,64.8845,156.3390,66.1195,156.2740,507.702495,0.039600,0.000989,0.011991,0.947420,2.493286e-12,2.246426e-12,2.246426e-12,4.063240e-07
315687,66.4943,154.1690,65.2593,154.3350,65.2902,156.3180,66.5267,156.2520,314.966702,0.012237,0.001662,0.005510,0.977523,4.018988e-12,1.519566e-06,1.715437e-03,1.351892e-03
315688,67.7118,153.9820,66.4748,154.1670,66.5073,156.2520,67.7458,156.1790,382.856310,0.010796,0.030397,0.000002,0.950429,3.306325e-12,4.279098e-05,8.333622e-03,2.978966e-12
315689,71.7666,153.1900,70.5228,153.4620,70.5622,155.9830,71.8085,155.8800,463.933604,0.005329,0.021477,0.030706,0.937292,2.728510e-12,2.899923e-03,1.015364e-03,1.281131e-03


In [48]:
# Load the base map
lunar_map_path = '/content/drive/My Drive/lunar_map_resized_big.png'
Image.MAX_IMAGE_PIXELS = None
lunar_map_pil = Image.open(lunar_map_path)
lunar_map = np.array(lunar_map_pil)
lunar_map = cv2.cvtColor(lunar_map, cv2.COLOR_RGB2BGR)

In [49]:
# Constants
min_lat, max_lat = -90, 90
min_lon, max_lon = -180, 180
height, width, _ = lunar_map.shape
opacity = 1.0

In [62]:
# Normalize the 'elemental_ratio' values
from matplotlib.cm import ScalarMappable
norm = Normalize(vmin=new_df['Ca_area_ratio'].min(), vmax=new_df['Ca_area_ratio'].max())

colormap = get_cmap('viridis')
scalar_map = ScalarMappable(norm=norm, cmap=colormap)

<ipython-input-62-761a10664c28>:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = get_cmap('viridis')


In [63]:
max_opacity=1.0
min_opacity=0.0
def lat_lon_to_pixel(lat, lon, img_width, img_height):
    x = int(((lon - min_lon) / (max_lon - min_lon)) * img_width)
    y = int(((max_lat - lat) / (max_lat - min_lat)) * img_height)
    return x, y

# Overlay the heatmap boxes
overlay = lunar_map.copy()
for _, row in new_df.iterrows():
    ul_x, ul_y = lat_lon_to_pixel(float(row['V0_lat']), float(row['V0_lon']), width, height)
    ur_x, ur_y = lat_lon_to_pixel(float(row['V1_lat']), float(row['V1_lon']), width, height)
    ll_x, ll_y = lat_lon_to_pixel(float(row['V2_lat']), float(row['V2_lon']), width, height)
    lr_x, lr_y = lat_lon_to_pixel(float(row['V3_lat']), float(row['V3_lon']), width, height)

    # Get box color
    normalized_value = norm(row['Ca_area_ratio'])
    color = scalar_map.to_rgba(row['Ca_area_ratio'], bytes=True)[:3]
    box_color = tuple(int(c) for c in color)
    d_opacity = min_opacity + normalized_value*(max_opacity - min_opacity)

    cv2.line(overlay, (ul_x, ul_y), (ur_x, ur_y), box_color, 2)
    cv2.line(overlay, (ur_x, ur_y), (lr_x, lr_y), box_color, 2)
    cv2.line(overlay, (lr_x, lr_y), (ll_x, ll_y), box_color, 2)
    cv2.line(overlay, (ll_x, ll_y), (ul_x, ul_y), box_color, 2)

# Blend the overlay with the base map
blended_image = np.zeros_like(lunar_map)
cv2.addWeighted(overlay, opacity, lunar_map, 1 - opacity, 0, blended_image)
lunar_map = blended_image[:, :, ::-1]  # Convert BGR to RGB for visualization

In [64]:
# Create the PNG with the color bar
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(lunar_map)
ax.axis('off')

# Add the color bar
cbar_ax = fig.add_axes([0.92, 0.25, 0.02, 0.5])  # Adjust position
cb = plt.colorbar(scalar_map, cax=cbar_ax)
cb.set_label('Elemental Ratio')

# Save the PNG
output_png_path = '/content/drive/My Drive/Ca_coverage4_heatmap.png'
plt.savefig(output_png_path, dpi=300, bbox_inches='tight')
plt.close()
